<a href="https://colab.research.google.com/github/Rishit02/FPL/blob/master/FPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credit to: https://towardsdatascience.com/fantasy-premier-league-value-analysis-python-tutorial-using-the-fpl-api-8031edfe9910#a7da

Import the relevant modules

In [19]:
import requests
import pandas as pd
import numpy as np

In [20]:
# The API url is:
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
# Using the requests package to make a GET request from the API endpoint:
r = requests.get(url)
# Transforming that request into a json object
json = r.json()
print(json.keys())
# Building dataframes
elements_df = pd.DataFrame(json["elements"])
elements_types_df = pd.DataFrame(json["element_types"])
teams_df = pd.DataFrame(json["teams"])
# print("Elements head", elements_df.head()) # Method
# print("Elements columns", elements_df.columns) # Method
s_elements_df = elements_df[['second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]
s_elements_df.head() # Method


dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])


,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points
0,Özil,1,3,1.1,70,0,834,0.0,0
1,Papastathopoulos,1,2,0.1,50,0,44,0.0,0
2,Luiz Moreira Marinho,1,2,0.8,55,0,345,0.0,0
3,Aubameyang,1,3,46.3,120,90,97062,0.6,7
4,Soares,1,2,0.5,49,0,670,0.0,0


Manipulating and sorting the data in the dataframe

Note: value = points / cost


In [21]:
# Adding the team name and the position
s_elements_df['position'] = s_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)
s_elements_df['team'] = s_elements_df.team.map(teams_df.set_index('id').name)
s_elements_df['value'] = s_elements_df.value_season.astype(float)
del s_elements_df["value_season"]
print("FILTERED ACCORDING TO VALUES (values = cost / points)")
s_elements_df.sort_values("value", ascending=False).head(25)


FILTERED ACCORDING TO VALUES (values = cost / points)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,total_points,position,value
27,Magalhães,Arsenal,2,9.1,50,90,313912,15,Defender,3.0
511,Saïss,Wolves,2,5.2,50,90,117120,15,Defender,3.0
372,Hendrick,Newcastle,3,3.4,50,90,161523,14,Midfielder,2.8
131,James,Chelsea,2,8.2,50,90,166127,14,Defender,2.8
241,Castagne,Leicester,2,4.0,55,90,117124,14,Defender,2.5
114,Frello Filho,Chelsea,3,4.3,50,84,120355,12,Midfielder,2.4
170,Digne,Everton,2,11.7,60,90,223846,12,Defender,2.0
239,Justin,Leicester,2,15.4,45,90,239515,9,Defender,2.0
140,Guaita,Crystal Palace,1,2.9,50,90,40319,10,Goalkeeper,2.0
26,Borges Da Silva,Arsenal,3,15.1,81,74,339529,14,Midfielder,1.7


Creating a weightage which will allow to choose players with high value AND greater

In [22]:
s_elements_df['total_points'] = s_elements_df.total_points.astype(float)
s_elements_df['weighted_assignments'] = (s_elements_df['value'] * 0.6) + s_elements_df['total_points'] * 0.4
print("FILTERED ACCORDING TO WEIGHTED ASSIGNMENTS")
s_elements_df.sort_values("weighted_assignments", ascending=False).head(25)


FILTERED ACCORDING TO WEIGHTED ASSIGNMENTS


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,total_points,position,value,weighted_assignments
281,Salah,Liverpool,3,36.4,120,90,252508,20.0,Midfielder,1.7,9.02
511,Saïss,Wolves,2,5.2,50,90,117120,15.0,Defender,3.0,7.80
27,Magalhães,Arsenal,2,9.1,50,90,313912,15.0,Defender,3.0,7.80
131,James,Chelsea,2,8.2,50,90,166127,14.0,Defender,2.8,7.28
372,Hendrick,Newcastle,3,3.4,50,90,161523,14.0,Midfielder,2.8,7.28
241,Castagne,Leicester,2,4.0,55,90,117124,14.0,Defender,2.5,7.10
26,Borges Da Silva,Arsenal,3,15.1,81,74,339529,14.0,Midfielder,1.7,6.62
114,Frello Filho,Chelsea,3,4.3,50,84,120355,12.0,Midfielder,2.4,6.24
170,Digne,Everton,2,11.7,60,90,223846,12.0,Defender,2.0,6.00
225,Vardy,Leicester,4,22.9,100,90,216747,13.0,Forward,1.3,5.98
